In [1]:
!pip install transformers torch

In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.3 MB/s eta 0:00:00


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
# Check if GPU is available and use it; otherwise, use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the tokenizer and the model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
class CodeSummarizationDataset(Dataset):
    def __init__(self, tokenizer, df, max_input_length, max_target_length):
        self.tokenizer = tokenizer
        self.input_texts = df['code'].tolist()
        self.target_texts = df['summary'].tolist()
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        source_text = self.input_texts[idx]
        target_text = self.target_texts[idx]

        source = self.tokenizer.encode_plus(
            f"summarize: {source_text}",
            max_length=self.max_input_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        target = self.tokenizer.encode_plus(
            target_text,
            max_length=self.max_target_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return source, target


In [7]:
df = pd.read_csv('/content/java-train_clean.tsv', sep='\t', error_bad_lines=False)

<ipython-input-7-274e47e44c7e>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/java-train_clean.tsv', sep='\t', error_bad_lines=False)


In [8]:
# Load your dataset
df.columns = ['code', 'summary']
train_df, test_df = train_test_split(df, test_size=0.2)

# Create Dataset and DataLoader
train_dataset = CodeSummarizationDataset(tokenizer, train_df, max_input_length=512, max_target_length=128)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [9]:
import time

# Set the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    start_time = time.time()

    total_loss = 0
    for batch in train_loader:
        input_ids = batch[0]['input_ids'].squeeze().to(device)
        attention_mask = batch[0]['attention_mask'].squeeze().to(device)
        labels = batch[1]['input_ids'].squeeze().to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    end_time = time.time()
    epoch_time = end_time - start_time
    total_time_remaining = (num_epochs - (epoch + 1)) * epoch_time

    print(f"Epoch {epoch} completed. Loss: {total_loss / len(train_loader)}")
    print(f"Time taken for epoch: {epoch_time // 60}m {epoch_time % 60}s")
    print(f"Estimated time remaining: {total_time_remaining // 60}m {total_time_remaining % 60}s")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0 completed. Loss: 0.6257788245494549
Time taken for epoch: 11.0m 6.798047780990601s
Estimated time remaining: 100.0m 1.1824300289154053s
Epoch 1 completed. Loss: 0.41608238473969367
Time taken for epoch: 11.0m 6.950345516204834s
Estimated time remaining: 88.0m 55.60276412963867s
Epoch 2 completed. Loss: 0.3909636334375298
Time taken for epoch: 11.0m 8.361825942993164s
Estimated time remaining: 77.0m 58.53278160095215s
Epoch 3 completed. Loss: 0.374083336302393
Time taken for epoch: 11.0m 7.0340588092803955s
Estimated time remaining: 66.0m 42.20435285568237s
Epoch 4 completed. Loss: 0.36171186830976915
Time taken for epoch: 11.0m 7.594344854354858s
Estimated time remaining: 55.0m 37.97172427177429s
Epoch 5 completed. Loss: 0.3508856801083929
Time taken for epoch: 11.0m 7.857542991638184s
Estimated time remaining: 44.0m 31.430171966552734s
Epoch 6 completed. Loss: 0.342417177542135
Time taken for epoch: 11.0m 9.103193998336792s
Estimated time remaining: 33.0m 27.309581995010376s
E

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
model.save_pretrained("/content/drive/My Drive/412 MODEL/code_summary_model.h5")

In [28]:
from transformers import T5ForConditionalGeneration

# Path to the saved model
load_path = '/content/drive/My Drive/412 MODEL/code_summary_model.h5'

# Create a new instance of the T5 model
loaded_model = T5ForConditionalGeneration.from_pretrained(load_path, local_files_only=True)

# Move the loaded model to the desired device (e.g., GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model.to(device)

# Now, your loaded_model is ready for further use.

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [43]:
def summarize_code(model, tokenizer, code, device, max_length=128):
    model.eval()
    input_ids = tokenizer.encode("summarize: " + code, return_tensors="pt", max_length=512, truncation=True).to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2)

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# Example usage
code_snippet = input("enter Java code: ")
summary = summarize_code(model, tokenizer, code_snippet, device)
print("Generated Summary:", summary)

enter Java code: public void createAccount(String accountNumber, double initialBalance) {         if (!accounts.containsKey(accountNumber)) {             accounts.put(accountNumber, initialBalance);             System.out.println("Account created successfully!");         } else {             System.out.println("Account already exists!");         }     }
Generated Summary: Creates a new account.


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18843 entries, 0 to 18842
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   code     18843 non-null  object
 1   summary  18842 non-null  object
dtypes: object(2)
memory usage: 294.5+ KB
